In [ ]:
library(dplyr)
library(readr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
load("/content/36151-0001-ind.rda")
ind <- da36151.0001

In [ ]:
load("/content/36151-0002-hh.rda")
hh<- da36151.0001

In [ ]:
columns1 <- c("IDHH", "RO3", "ED5", "HHEDUCF", "HHEDUCM", "CS4", "CS5", "CS6", "CS22", "CS24", "CS25", "CS26","CS27", "CS28")
ind <- ind %>% select(all_of(columns1))

In [ ]:
columns2 <- c("IDHH", "INCOME", "URBAN2011", "ID11", "ID13","STATEID","NCHILDM","NCHILDF","NTEENM","NTEENF","ASSETS")
hh<- hh%>% select(all_of(columns2))

In [ ]:
ind <- ind %>% filter(!is.na(CS25) & !is.na(CS26))
ind <- ind %>%
  mutate(CS27 = ifelse(is.na(CS27), 0, CS27),
         CS28 = ifelse(is.na(CS28), 0, CS28))

In [ ]:
ind <- ind %>%
  mutate(ind_edu_exp = CS25 + CS26 + CS27 + CS28)

In [ ]:
hh <- hh %>%
  mutate(NCHILD = NCHILDM + NCHILDF + NTEENM + NTEENF)

In [ ]:
ind <- ind %>%
  filter(ED5 == "(1) Yes 1" & RO3 == "(2) Female 2")

In [ ]:
ind <- ind %>%
  group_by(IDHH) %>%
  mutate(female_count = sum(RO3 == "(2) Female 2"))

In [ ]:
ind <- ind %>%
  select(-RO3,-ED5)

In [ ]:
ind <- ind %>%
  group_by(IDHH) %>%
  mutate(female_edu_exp = sum(ind_edu_exp, na.rm = TRUE)) %>%
  ungroup()

In [ ]:
hh <- hh %>% filter(!is.na(ASSETS))

In [ ]:
hh_ind <- inner_join(hh, ind, by = "IDHH", relationship = "many-to-many")

In [ ]:
#merged dataset
hh_ind <- hh_ind %>% distinct()

In [ ]:
nrow(hh_ind)

[1] 24416

In [ ]:
write_csv(hh_ind, "hh_ind.csv")

In [ ]:
extract_numeric <- function(x) {
  as.numeric(gsub(".*\\((\\d+)\\).*", "\\1", x))
}

# List of columns to clean
cols_to_clean <- c("URBAN2011", "ID11", "ID13", "HHEDUCF", "HHEDUCM","CS4","CS6","CS22","STATEID")

# Apply the function to the selected columns
hh_ind[cols_to_clean] <- lapply(hh_ind[cols_to_clean], extract_numeric)

In [ ]:
hh_ind <- hh_ind %>%
  rename(
  HH_INCOME = INCOME,
    RELIGION = ID11,
    CASTE = ID13,
    SCHOOL_TYPE = CS4,
    SCHOOL_DIST = CS5,
    STANDARD = CS6,
    FEES_BY_GOVT_DUMMY = CS22,
    SCHOLARSHIP = CS24,
    )

In [ ]:
hh_ind <- hh_ind %>%
  select(-CS25,-CS26,-CS27,-CS28,-NCHILDM,-NCHILDF,-NTEENM,-NTEENF)

In [ ]:
head(hh_ind)

,IDHH,HH_INCOME,URBAN2011,RELIGION,CASTE,STATEID,ASSETS,NCHILD,HHEDUCF,HHEDUCM,SCHOOL_TYPE,SCHOOL_DIST,STANDARD,FEES_BY_GOVT_DUMMY,SCHOLARSHIP,ind_edu_exp,female_count,female_edu_exp
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,0102010101,176100,0,2,3,1,20,5,8,9,4,3,6,0,0,10600,2,13400
2,0102010101,176100,0,2,3,1,20,5,8,9,4,3,0,0,0,2800,2,13400
3,0102010201,1039150,0,2,NA,1,24,8,12,16,2,3,9,0,0,3200,4,120200
4,0102010201,1039150,0,2,NA,1,24,8,12,16,4,24,3,0,0,39000,4,120200
5,0102010201,1039150,0,2,NA,1,24,8,12,16,4,24,2,0,0,39000,4,120200
6,0102010301,182340,0,2,3,1,22,4,11,10,4,3,10,0,0,3900,2,6580


In [ ]:
# Function to map education levels to years of education
convert_to_years <- function(x) {
  case_when(
    is.na(x) ~ 0,          # Treat NA as no education
    x == 0 ~ 0,            # No education
    x == 55 ~ 0,           # Special case for <1 class
    x >= 1 & x <= 14 ~ x,  # School years map directly
    x == 15 ~ 19,          # Bachelor's degree (16 -> 19 years)
    x == 16 ~ 21,          # Above Bachelor's (Assuming Master's)
    TRUE ~ NA_real_        # Keep any unexpected values as NA
  )
}

# Apply the function to HHEDUCF and HHEDUCM
hh_ind <- hh_ind %>%
  mutate(
    HHEDUCF_YEARS = convert_to_years(HHEDUCF),
    HHEDUCM_YEARS = convert_to_years(HHEDUCM),
    STANDARD = convert_to_years(STANDARD)
  )


In [ ]:
hh_ind <- hh_ind %>%
  select(-HHEDUCF,-HHEDUCM)

In [ ]:
hh_ind <- hh_ind %>%
  filter(!is.na(HHEDUCF_YEARS) & !is.na(HHEDUCM_YEARS) & !is.na(STANDARD))

In [ ]:
hh_ind <- hh_ind %>%
  mutate(
    SCHOLARSHIP = ifelse(is.na(SCHOLARSHIP), 0, SCHOLARSHIP),
    SCHOOL_DIST = ifelse(is.na(SCHOOL_DIST), 0, SCHOOL_DIST)
  )

In [ ]:
hh_ind <- hh_ind %>%
  filter(!is.na(CASTE))

In [ ]:
# Recode the religion variable into 4 categories
hh_ind <- hh_ind %>%
  mutate(
    RELIGION_GROUPED = case_when(
      RELIGION == 1 ~ "Hindu",
      RELIGION == 2 ~ "Muslim",
      RELIGION %in% c(3, 4) ~ "Christian_Sikh",
      RELIGION %in% c(5, 6, 7, 8, 9) ~ "Other"
    )
  )

# Convert to factor
hh_ind$RELIGION_GROUPED <- as.factor(hh_ind$RELIGION_GROUPED)

In [ ]:
#HINDU is the reference category
hh_ind<- hh_ind%>%
  mutate(
    RELIGION_MUSLIM = ifelse(RELIGION_GROUPED == "Muslim", 1, 0),
    RELIGION_CHRISTIAN_SIKH = ifelse(RELIGION_GROUPED == "Christian_Sikh", 1, 0),
    RELIGION_OTHER = ifelse(RELIGION_GROUPED == "Other", 1, 0)
  )

In [ ]:
# Recode the caste variable into 3 categories
hh_ind<- hh_ind%>%
  mutate(
    CASTE_GROUPED = case_when(
      CASTE %in% c(1,2) ~ "General",
      CASTE == 3 ~ "OBC",
      CASTE %in% c(4,5,6) ~ "SC/ST/Others"
    )
  )

# Convert to factor
hh_ind$CASTE_GROUPED <- as.factor(hh_ind$CASTE_GROUPED)

In [ ]:
#OBC is reference category
hh_ind<- hh_ind%>%
  mutate(
    CASTE_GENERAL = ifelse(CASTE_GROUPED == "General", 1, 0),
    CASTE_SC_ST_OTHERS= ifelse(CASTE_GROUPED == "SC/ST/Others", 1, 0)
  )

In [ ]:
# hh_ind <- hh_ind %>%
#   filter(SCHOOL_TYPE %in% c(2, 3, 4))

In [ ]:
# Recode the caste variable into 3 categories
# hh_ind<- hh_ind%>%
#   mutate(
#     SCHOOL_TYPE_GROUPED = case_when(
#       SCHOOL_TYPE == 2 ~ "GOVERNMENT",
#       SCHOOL_TYPE == 3 ~ "GOVT_AIDED",
#       SCHOOL_TYPE == 4 ~ "PRIVATE"
#     )
#   )

# # Convert to factor
# hh_ind$SCHOOL_TYPE_GROUPED <- as.factor(hh_ind$SCHOOL_TYPE_GROUPED)

In [ ]:
# hh_ind<- hh_ind%>%
#   mutate(
#     SCHOOL_PRIVATE = ifelse(SCHOOL_TYPE_GROUPED == "PRIVATE", 1, 0),
#     SCHOOL_GOVT_AIDED = ifelse(SCHOOL_TYPE_GROUPED == "GOVT_AIDED", 1, 0)
#   )

In [ ]:
hh_ind <- hh_ind %>%
  select(-RELIGION_GROUPED,-CASTE_GROUPED,-RELIGION,-CASTE,-SCHOOL_TYPE,-SCHOOL_DIST,-FEES_BY_GOVT_DUMMY,-SCHOLARSHIP,-STATEID)

In [ ]:
head(hh_ind)

,IDHH,HH_INCOME,URBAN2011,ASSETS,NCHILD,STANDARD,ind_edu_exp,female_count,female_edu_exp,HHEDUCF_YEARS,HHEDUCM_YEARS,RELIGION_MUSLIM,RELIGION_CHRISTIAN_SIKH,RELIGION_OTHER,CASTE_GENERAL,CASTE_SC_ST_OTHERS
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0102010101,176100,0,20,5,6,10600,2,13400,8,9,1,0,0,0,0
2,0102010101,176100,0,20,5,0,2800,2,13400,8,9,1,0,0,0,0
3,0102010301,182340,0,22,4,10,3900,2,6580,11,10,1,0,0,0,0
4,0102010301,182340,0,22,4,5,2680,2,6580,11,10,1,0,0,0,0
5,0102010401,90760,0,16,3,8,1150,1,1150,0,0,1,0,0,1,0
6,0102010601,152100,0,12,4,0,220,1,220,9,0,1,0,0,0,0


In [ ]:
save(hh_ind, file = "hh_ind.rda")
write.csv(hh_ind,"hh_ind.csv")

In [ ]:
nrow(hh_ind)

[1] 24372